# 뉴스 토픽 분류 경진대회
- 2021.06.30 ~ 2021.08.09
- `public score: 0.8637`, `private score : 0.83004`
- 사용 모델 :bert 계열(kobert, koelectra, xlm-robera)
- task : 텍스트 분류
koelectra를 이용하여 k-fold(k = 3)일때 성능이 가장 좋음.

## 라이브러리 로딩

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import pandas as pd
import numpy as np
from tqdm import tqdm
from transformers import AdamW

from sklearn.model_selection import train_test_split
import os
import random
import time
import datetime


# GPU 사용 시
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [5]:
def seed_everything(seed: int = 42):
    """
    seed 고정
    """
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True


seed_everything()

In [6]:
# 데이터 불러오기
train = pd.read_csv('train_data.csv')
test = pd.read_csv('test_data.csv')

In [7]:
train.head()

,index,title,topic_idx
0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화,4
2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4
3,3,NYT 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합,4
4,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4


## 텍스트 데이터 전처리

In [ ]:
# 성능이 오히려 하락했기 떄문에 적용하지 않음
'''
from konlpy.tag import Okt
# 형태소 분석기(Okt) 불러오기 
okt=Okt() 

# 조사, 어미, 구두점 제거
def func(text):
    clean = []
    for word in okt.pos(text, stem=True): #어간 추출
        if word[1] not in ['Josa', 'Eomi', 'Punctuation']: #조사, 어미, 구두점 제외 
            clean.append(word[0])
    
    
    return " ".join(clean) 

train['title'] = train['title'].apply(lambda x : func(x))
test['title'] = test['title'].apply(lambda x : func(x))
'''

## 모델링

In [9]:
class dataset(Dataset):
    def __init__(self, dataset, max_len, bert_tokenizer):

        self.tokenizer = bert_tokenizer
        self.dataset = dataset
        self.max_len = max_len

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        #row = self.dataset.iloc[idx, :2].values
        text = self.dataset.iloc[idx, 0]
        label = self.dataset.iloc[idx, 1]

        inputs = self.tokenizer(
            text,
            return_tensors='pt',
            truncation=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            add_special_tokens=True
        )

        input_ids = inputs['input_ids'][0]
        attention_mask = inputs['attention_mask'][0]

        return input_ids, attention_mask, label

In [10]:
train.head()

,index,title,topic_idx
0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화,4
2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4
3,3,NYT 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합,4
4,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4


In [11]:
def get_split_data(df):
    """
    train 데이터를 8:2 비율로 train과 valid 데이터로 나누는 함수
    """

    X, y = df[['title']], df['topic_idx']
    train_x, valid_x, train_y, valid_y = train_test_split(
        X, y, stratify=y, test_size=0.2)

    train_x['topic_idx'] = train_y
    valid_x['topic_idx'] = valid_y

    return train_x, valid_x


def get_dataloader(df, mode, batch_size, max_len, tok):
    """
    데이터프레임을 dataloader형태로 반환하는 함수
    """

    if mode == 'TRAIN':
        train, valid = get_split_data(df)

        train_dataset = dataset(train, max_len, tok)
        valid_dataset = dataset(valid, max_len, tok)

        train_loader = DataLoader(
            train_dataset, batch_size=batch_size, shuffle=True)
        valid_loader = DataLoader(
            valid_dataset, batch_size=batch_size, shuffle=True)

        return train_loader, valid_loader
    else:
        test_dataset = dataset(test, max_len, tok)
        test_loader = DataLoader(
            test_dataset, batch_size=batch_size, shuffle=False)
        return test_loader

In [12]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

modelname = 'monologg/koelectra-base-v3-discriminator'  # koelectra 모델 사용
# modelname = 'xlm-roberta-large' # roberta large 모델 사용

tokenizer = AutoTokenizer.from_pretrained(modelname)
train_loader, val_loader = get_dataloader(
    train, mode='TRAIN', batch_size=8, max_len=50, tok=tokenizer)
#  dict 형식으로 data loader 정의
dataloaders_dict = {"train": train_loader, "val": val_loader}

In [17]:
def train_model(model, dataloaders_dict, optimizer, num_epochs):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print('사용하는 device :', device)
    print('----------start----------')

    model.to(device)
    torch.cuda.empty_cache()
    # 학습이 어느정도 진행되면 gpu 가속화
    #torch.backends.cudnn.benchmark = False

    # loss가 제일 낮은 모델을 찾기위한 변수
    best_val_loss = int(1e9)
    for epoch in range(num_epochs):
        # epoch 별 학습 및 검증

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # 모델을 학습 모드로
            else:
                model.eval()   # 모델을 추론 모드로

            epoch_loss = 0.0  # epoch loss
            epoch_corrects = 0  # epoch 정확도
            for i,  (input_ids_batch, attention_masks_batch, y_batch) in enumerate(dataloaders_dict[phase]):
                input_ids_batch = input_ids_batch.to(device)
                attention_masks_batch = attention_masks_batch.to(device)
                y_batch = y_batch.to(device)

                # 옵티마이저 초기화 초기화
                optimizer.zero_grad()

                # 순전파 계산
                with torch.set_grad_enabled(phase == 'train'):
                    y_pred = model(input_ids_batch, attention_masks_batch)[0]
                    loss = F.cross_entropy(y_pred, y_batch)

                    # 학습시 역전파
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    # 결과 계산
                    # loss계산
                    epoch_loss += loss.item() * len(input_ids_batch)
                    # 정확도 계산
                    epoch_corrects += (torch.argmax(y_pred, axis=1)
                                       == y_batch.data).sum().item()

            # epoch별 loss 및 정확도

            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = 100 * epoch_corrects / \
                len(dataloaders_dict[phase].dataset)

            print('Epoch {}/{} | {:^5} |  Loss: {:.4f} Acc: {:.4f}'.format(epoch+1, num_epochs,
                                                                           phase, epoch_loss, epoch_acc))

            # 검증 오차가 가장 적은 최적의 모델을 저장
            if not best_val_loss or epoch_loss < best_val_loss:
                best_val_loss = epoch_loss
                best_model = model

    return best_model

In [ ]:
best_models = []
# 교차 검증을 진행할 K
K = 1
for i in range(K):
    num_epochs = 5
    max_grad_norm = 1
    learning_rate = 5e-5

    model = AutoModelForSequenceClassification.from_pretrained(
        modelname, num_labels=7).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

    best_model = train_model(model, dataloaders_dict, optimizer, num_epochs)
    best_models.append(best_model)

## 추론 및 정답파일 생성

In [ ]:
len(best_models)

5

In [ ]:
test['topic_label'] = 0
test_dataset = get_dataloader(
    test, mode='TEST', batch_size=16, max_len=50, tok=tokenizer)

In [ ]:
# 평가모드로 변경
preds = []
for idx, Best_Model in enumerate(best_models):
    print('#'*10, idx+1, ": 번째 모델 예측 진행", '#'*10)
    model = Best_Model
    model.eval()

    pred = []
    for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
        y_batch = y_batch.cuda()
        y_pred = model(input_ids_batch.cuda(),
                       attention_mask=attention_masks_batch.cuda())[0]
        pred.extend(y_pred.cpu().detach().numpy().tolist())
    preds.append(pred)

########## 1 : 번째 모델 예측 진행 ##########


  0%|          | 0/9131 [00:00<?, ?it/s]

########## 2 : 번째 모델 예측 진행 ##########


  0%|          | 0/9131 [00:00<?, ?it/s]

########## 3 : 번째 모델 예측 진행 ##########


  0%|          | 0/9131 [00:00<?, ?it/s]

########## 4 : 번째 모델 예측 진행 ##########


  0%|          | 0/9131 [00:00<?, ?it/s]

########## 5 : 번째 모델 예측 진행 ##########


  0%|          | 0/9131 [00:00<?, ?it/s]

In [ ]:
new_pred = []
for i in range(len(preds[0])):
    a = np.array(preds[0][i])
    b = np.array(preds[1][i])
    c = np.array(preds[2][i])
    d = np.array(preds[3][i])
    e = np.array(preds[4][i])

    new_pred.append(a+b+c+d+e)

len(new_pred)

9131

In [ ]:
pred = np.argmax(new_pred,1)
pred

array([0, 3, 2, ..., 2, 2, 2])

In [ ]:
sub = pd.read_csv('sample_submission.csv')
sub.topic_idx = pred